In [2]:
%matplotlib inline

import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt

from qiime.parse import parse_mapping_file
from qiime.format import format_mapping_file
from skbio.io.util import open_file
from scipy.stats import pearsonr, spearmanr

def load_mf(fn):
    with open_file(fn, 'U') as f:
        mapping_data, header, _ = parse_mapping_file(f)
        _mapping_file = pd.DataFrame(mapping_data, columns=header)
        _mapping_file.set_index('SampleID', inplace=True)
    return _mapping_file

def write_mf(f, _df):
    with open_file(f, 'w') as fp:
        lines = format_mapping_file(['SampleID'] + _df.columns.tolist(),
                                    list(_df.itertuples()))
        fp.write(lines+'\n')

# Obtaining the metadata and removing unused data for the analysis

We have a mapping file that we used for an overall initial analysis, see `Analysis.ipynb`, however we added more metadata (namely `indiv_g_protein_1000kcal_ME_group` and `indiv_g_fat_1000kcal_ME_group`) after that inital analysis was completed. Thus, in trying to ensure `Analysis.ipynb` continues to work, we'll use a new mapping file named **`mapping-file-full.txt`**. I received this file from Jan on October 8.

In [2]:
ls mapping-file-full.txt

mapping-file-full.txt


In [3]:
new_mf = load_mf('mapping-file-full.txt')
old_mf = load_mf('mapping-file.txt')

In [4]:
set(new_mf.columns) - set(old_mf.columns)

{'indiv_g_fat_1000kcal_ME_group', 'indiv_g_protein_1000kcal_ME_group'}

In [5]:
print len(new_mf.columns)

90


In [6]:
print len(old_mf.columns)

88


In [7]:
print len(new_mf), len(old_mf)

192 192


In [8]:
a = new_mf.copy()
b = old_mf.copy()

for column_name in set(a.columns) & set(b.columns):
    try:
        pd.util.testing.assert_series_equal(a[column_name], b[column_name])
    except AssertionError as e:
        print column_name, 'changed from the old to the new file', str(e)

RUN_DATE changed from the old to the new file '1/30/2012' != '1/30/12'
collection_date changed from the old to the new file '1/30/2012' != '1/30/12'
Weight_group changed from the old to the new file '3' != '3.0'
Weight_kg changed from the old to the new file '31' != '31.0'
tot_mass changed from the old to the new file '31' != '31.0'
BCS changed from the old to the new file '8' != '8.0'
elevation changed from the old to the new file '571' != '571.0'
COLLECTION_TIMESTAMP changed from the old to the new file '1/30/2012 0:00' != '1/30/12 0:00'
Age_group changed from the old to the new file '3' != '3.0'
age changed from the old to the new file '8' != '8.0'


Most of these changes seem to be Excel coercing data into a different data type, since this is the case I'll just add the new columns to the old mapping file and serialize that.

In [9]:
!mv -n mapping-file-full.txt mapping-file-excelified.txt

In [10]:
new_mf = load_mf('mapping-file-excelified.txt')
old_mf = load_mf('mapping-file.txt')

old_mf['indiv_g_fat_1000kcal_ME_group'] = new_mf['indiv_g_fat_1000kcal_ME_group'].copy()
old_mf['indiv_g_protein_1000kcal_ME_group'] = new_mf['indiv_g_protein_1000kcal_ME_group'].copy()

# this cell had a value of 558.8, which I confirmed with Jan and the correct value should be 58.8
old_mf = old_mf.set_value('Nor.C1', 'indiv_g_protein_1000kcal_ME_group', '58.8')

write_mf('mapping-file-full.txt', old_mf)

In [11]:
old_mf.disease_stat.value_counts()

healthy                98
IBD                    79
acute hem. diarrhea    15
dtype: int64

In [44]:
!filter_samples_from_otu_table.py -i otu_table.15000.biom \
-o otu_table.15000.no-diarrhea.biom \
-s 'disease_stat:!acute hem. diarrhea,*' \
-m mapping-file-full.txt

In [12]:
%%bash 

echo 'Without filtering'
biom summarize-table -i otu_table.15000.biom | head
echo 'After filtering'
biom summarize-table -i otu_table.15000.no-diarrhea.biom | head

Without filtering
Num samples: 162
Num observations: 5467
Total count: 2430000
Table density (fraction of non-zero values): 0.052

Counts/sample summary:
 Min: 15000.0
 Max: 15000.0
 Median: 15000.000
 Mean: 15000.000
After filtering
Num samples: 149
Num observations: 5467
Total count: 2235000
Table density (fraction of non-zero values): 0.053

Counts/sample summary:
 Min: 15000.0
 Max: 15000.0
 Median: 15000.000
 Mean: 15000.000


# Add $\alpha$-diversity information

In [13]:
!add_alpha_to_mapping_file.py -m mapping-file-full.txt \
--alpha_fps alpha/alpha-20000/alpha_div_collated/PD_whole_tree.txt,\
alpha/alpha-20000/alpha_div_collated/chao1.txt,\
alpha/alpha-20000/alpha_div_collated/observed_species.txt,\
alpha/alpha-20000/alpha_div_collated/shannon.txt \
-b 4 \
-o mapping-file-full.alpha.txt \
--depth 15000 \
--collated_input

#Add taxa summaries information

In [14]:
!summarize_taxa.py -i otu_table.15000.no-diarrhea.biom \
-o taxonomic_summaries/no-diarrhea/ \
-m mapping-file-full.alpha.txt

# Add Gevers et al dysbiosis index

In [15]:
mf = load_mf('taxonomic_summaries/no-diarrhea/mapping-file-full.alpha_L6.txt')

# according to figure 2 in the Gevers paper
prot = {'__Dialister', '__Bilophila', '__Oscillospira', '__Faecalibacterium', '__Ruminococcaceae', '__Dorea',
        '__Erysipelotrichaceae', '__Ruminococcus', '__Coprococcus', '__Lachnospiraceae', '__Bacteroides',
        '__Parabacteroides', '__Rikenellaceae', '__Sutterella'}
infl = {'__Fusobacterium', '__Haemophilus', '__Veillonella', '__Escherichia'}

mf['Gevers_Protective'] = pd.Series(np.zeros_like(mf.index.values), mf.index, dtype=np.float)
mf['Gevers_Inflammatory'] = pd.Series(np.zeros_like(mf.index.values), mf.index, dtype=np.float)
for column_name in mf.columns:
    if any([True for p in prot if p in column_name]):
        mf['Gevers_Protective'] += mf[column_name].astype(np.float)
    elif any([True for i in infl if i in column_name]):
        mf['Gevers_Inflammatory'] += mf[column_name].astype(np.float)
    else:
        continue

# calculating the dysbiosis index
mf['Gevers_Dysbiosis_Index'] = np.divide(mf['Gevers_Inflammatory'], mf['Gevers_Protective']).astype(np.float)
# drop any samples with undefined values
mf['Gevers_Dysbiosis_Index'].replace({0: np.nan}, inplace=True)
mf['Gevers_Dysbiosis_Index'] = np.log(mf['Gevers_Dysbiosis_Index'])

# convert the data to strings so we can serialize it
for column_name in ['Gevers_Dysbiosis_Index', 'Gevers_Protective', 'Gevers_Inflammatory']:
    mf[column_name] = mf[column_name].astype(str)
    
write_mf('mapping-file-full.alpha.L6index.txt', mf)
del mf

In [3]:
mf = load_mf('mapping-file-full.alpha.L6index.txt')

In [6]:
mf.country.value_counts()

GAZ:United States of America    41
GAZ:Sweden                      27
GAZ:Brazil                      27
GAZ:France                      16
GAZ:Germany                     14
GAZ:United Kingdom              12
GAZ:Finland                     12
Name: country, dtype: int64

In [7]:
mf.disease_stat.value_counts()

healthy    85
IBD        64
Name: disease_stat, dtype: int64